# DAG Verification UI 
### consider drag users to the specific python file 

In [1]:
import yaml
import webbrowser
import subprocess
import time
import sys
import os

# Define DAG data
dag = {
    "nodes": [
        {
            "id": "driver_creation",
            "location": "data_pipeline.py:10-50",
            "inputs": {},
            "outputs": {
                "driver_path": "gs://my-bucket/data/driverset.parquet"
            }
        },
        {
            "id": "feature_pull",
            "location": "data_pipeline.py:60-100",
            "inputs": {
                "driver_path": "gs://my-bucket/data/driverset.parquet",
                "feature_list": "columns/features.txt"
            },
            "outputs": {
                "train_data_path": "gs://my-bucket/data/train_data.parquet",
                "test_data_path": "gs://my-bucket/data/test_data.parquet"
            }
        },
        {
            "id": "feature_selection",
            "location": "feature_engineering.py:10-50",
            "inputs": {
                "train_data_path": "gs://my-bucket/data/train_data.parquet"
            },
            "outputs": {
                "final_feature_list": "final_features/selected_features.txt"
            }
        },
        {
            "id": "data_normalization",
            "location": "feature_engineering.py:35-70",
            "inputs": {
                "final_feature_list": "final_features/selected_features.txt"
            },
            "outputs": {
                "normalized_train_data_path": "gs://my-bucket/data/processed/normalized_train_data.parquet"
            }
        },
        {
            "id": "model_training",
            "location": "train_model.py:20-80",
            "inputs": {
                "normalized_train_data_path": "gs://my-bucket/data/processed/normalized_train_data.parquet"
            },
            "outputs": {
                "model_path": "gs://my-bucket/models/model.onnx"
            }
        },
        {
            "id": "model_scoring",
            "location": "train_model.py:90-110",
            "inputs": {
                "model_path": "gs://my-bucket/models/model.onnx",
                "test_data_path": "gs://my-bucket/data/test_data.parquet"
            },
            "outputs": {
                "scored_test_data_path": "gs://my-bucket/data/scores/scored_test_data.parquet"
            }
        },
        {
            "id": "model_evaluation",
            "location": "train_model.py:120-150",
            "inputs": {
                "scored_test_data_path": "gs://my-bucket/data/scores/scored_test_data.parquet"
            },
            "outputs": {
                "evaluation_results_path": "gs://my-bucket/evaluation/model_performance.csv"
            }
        }
    ],
    "edges": [
        {
            "from": "driver_creation",
            "to": "feature_pull",
            "type": "data_dependency",
            "attributes": {
                "driver_path": "gs://my-bucket/data/driverset.parquet"
            }
        },
        {
            "from": "feature_pull",
            "to": "feature_selection",
            "type": "data_dependency",
            "attributes": {
                "train_data_path": "gs://my-bucket/data/train_data.parquet"
            }
        },
        {
            "from": "feature_pull",
            "to": "data_normalization",
            "type": "data_dependency",
            "attributes": {
                "train_data_path": "gs://my-bucket/data/train_data.parquet"
            }
        },
        {
            "from": "feature_selection",
            "to": "data_normalization",
            "type": "data_dependency",
            "attributes": {
                "final_feature_list": "final_features/selected_features.txt"
            }
        },
        {
            "from": "data_normalization",
            "to": "model_training",
            "type": "data_dependency",
            "attributes": {
                "normalized_train_data_path": "gs://my-bucket/data/processed/normalized_train_data.parquet"
            }
        },
        {
            "from": "model_training",
            "to": "model_scoring",
            "type": "data_dependency",
            "attributes": {
                "model_path": "gs://my-bucket/models/model.onnx"
            }
        },
        {
            "from": "feature_pull",
            "to": "model_scoring",
            "type": "data_dependency",
            "attributes": {
                "test_data_path": "gs://my-bucket/data/test_data.parquet"
            }
        },
        {
            "from": "model_scoring",
            "to": "model_evaluation",
            "type": "data_dependency",
            "attributes": {
                "scored_test_data_path": "gs://my-bucket/data/scores/scored_test_data.parquet"
            }
        }
    ]
}

# Save both the original and the modified files
original_path = "/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/backend/original_dag.yaml"
verified_path = "/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/backend/verified_yaml.yaml"


with open(original_path, "w") as file:
    yaml.dump(dag, file)

# verified_yaml.yaml is initially identical to the original file
with open(verified_path, "w") as file:
    yaml.dump(dag, file)

print("DAG YAML generated successfully, both original and modified files have been created!")

# Launch the React frontend for windows
# react_process = subprocess.Popen(
#     ["npm", "start"],
#     cwd=r"/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/frontend",  # React project directory
#     shell=True,
#     creationflags=subprocess.CREATE_NEW_CONSOLE
# )
# time.sleep(5)

# # Launch the FastAPI backend for windows
# backend_process = subprocess.Popen(
#     [sys.executable, "-m", "uvicorn", "server:app", "--reload"],
#     cwd=r"/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/backend",
#     shell=True,
#     creationflags=subprocess.CREATE_NEW_CONSOLE
# )
# time.sleep(1)

# # Open the frontend page
# webbrowser.open("http://localhost:3000")

# start frontend for macbook
frontend_script = '''
tell application "Terminal"
    do script "cd /Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/frontend && npm start"
end tell
'''
subprocess.Popen(["osascript", "-e", frontend_script])

time.sleep(5)

# start backend for macbook
backend_dir = "/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/backend"
os.environ["PYTHONPATH"] = backend_dir

backend_process = subprocess.Popen([
    "/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/.venv/bin/python",
    "-m", "uvicorn", "server:app", "--reload", "--host", "0.0.0.0", "--port", "8000"
], cwd=backend_dir)  # have to make sure the endpoint port is available

# wait for 3s
time.sleep(3)

# open UI
webbrowser.open("http://localhost:3000")



print("React frontend & FastAPI backend have been launched!")


DAG YAML generated successfully, both original and modified files have been created!
tab 1 of window id 3243


INFO:     Will watch for changes in these directories: ['/Users/yanfdai/Desktop/codespace/DAG_FULLSTACK/DAG_fullstack/backend']
ERROR:    [Errno 48] Address already in use


React frontend & FastAPI backend have been launched!
